In [2]:
import os
import glob
import subprocess
import shutil
import logging
from datetime import datetime

In [3]:
# configuration
INPUT_DIR= "/mnt/c/Users/bzwil/OneDrive/Desktop/KaliningradProject/kaliningrad-geoint-change-detection/files/data/prepared_data"
DB_NAME = "kaliningraddata"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
TABLE = "public.dem_clipped"
SRS = "4326"
TILE_SIZE="100x100"
LOG_FILE="/mnt/c/Users/bzwil/OneDrive/Desktop/KaliningradProject/kaliningrad-geoint-change-detection/files/transfer.log"

In [ ]:
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def find_raster2pgsql():
    possible_path = [
        "/usr/bin/raster2pgsql"
    ]
    
    for path in possible_path: 
        if os.path.isfile(path) and os.access(path, os.X_OK):
            return path
        else:
            print("your not correct brother")
        
    return shutil.which("raster2pgsql")